In [281]:
# ANN Training
import numpy as np
import pandas as pd
import random
import copy
import time
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import Dropout


In [ ]:
from keras.losses import binary_crossentropy

# Compute Binary Cross-Entropy using NumPy
def binary_cross_entropy_np(y_true, y_pred):
    """
    Calculates Binary Cross-Entropy loss for multiple samples using NumPy.
    y_true: NumPy array of actual labels (0s and 1s)
    y_pred: NumPy array of predicted probabilities (between 0 and 1)
    """
    epsilon = 1e-15  # Small value to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss



# Compute Binary Cross-Entropy using own calculaton for checking
def binary_cross_entropy_check(y_true, y_pred):
    """
    Calculates Binary Cross-Entropy loss for multiple samples using NumPy.
    y_true: NumPy array of actual labels (0s and 1s)
    y_pred: NumPy array of predicted probabilities (between 0 and 1)
    """
    epsilon = 1e-15  # Small value to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
    
    N=len(y_true)
    BCE_sum=0
    for i in range(N):
        BCE_sum = BCE_sum + y_true[i]*np.log(y_pred[i]) + (1-y_true[i])*np.log((1-y_pred[i])) 

    BCE = (-1*BCE_sum)/N
    return BCE


In [283]:
# To define some ANN models

def baseline_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim = 2000))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model


def baseline_model2():
    model = Sequential()
    model.add(Dense(256, activation='relu', input_dim = 2000))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

def baseline_model3():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim = 2000))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model


# Build the model
# ANN_model = baseline_model()

ANN_model = baseline_model2()

ANN_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 256)               512256    
_________________________________________________________________
batch_normalization_33 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 64)                16448     
_________________________________________________________________
batch_normalization_34 (Batc (None, 64)                256       
_________________________________________________________________
dropout_22 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 16)              

In [284]:
# To create the balanced training and testing dataset


In [ ]:
# To create the dataset
# Ref vector is 1x1000; same for variant vector
def generate_dataset(num_items,size_ref_vector,label_1_matrix,label_0_matrix,label_1_sign,label_0_sign):
    train_x = np.empty((0,2*size_ref_vector))
    train_y = np.empty((0,1))
    lowerB = 0.99
    upperB = 1.01
    lowerR = 1.2  # modify individual element of variant vector
    upperR = 1.5  # in range [1.2, 1.5]

    for i in range(0,num_items):
        #Ref vector
        ref_vector1 = np.array(np.random.uniform(low=0.01, high=0.6, size=size_ref_vector))
        # Initial variant is obtained from deepcopy
        k_adjust = random.uniform(lowerB,upperB)

        var_vector1 = copy.deepcopy(ref_vector1)*k_adjust

        index = (i % 80)  #label 1 40 cases, label 0 also 40 cases
        
        # Generate 40 lABEL 1 data points
        for icase in range(0,40):
            pos = np.array(label_1_matrix[icase,:])
            kk_sign = np.array(label_1_sign[icase,:])
            counter = 0
            for j in pos:
                kk = random.uniform(lowerR,upperR)   
                var_vector1[j] = ref_vector1[j]*kk*kk_sign[counter]
                # debug print('--- ', icase,j,kk,counter,kk_sign[counter])
                counter = counter + 1

            zzz_x = np.concatenate((ref_vector1,var_vector1))   
            zzz_y = np.array([1])
            train_x = np.append(train_x,[zzz_x],axis=0)
            train_y = np.append(train_y,[zzz_y],axis=0)

        # Generate 40 lABEL 0 data points
        for icase in range(0,40):
            pos = np.array(label_0_matrix[icase,:])
            kk_sign = np.array(label_0_sign[icase,:])
            counter = 0
            for j in pos:
                kk = random.uniform(lowerR,upperR)
                var_vector1[j] = ref_vector1[j]*kk*kk_sign[counter]

            zzz_x = np.concatenate((ref_vector1,var_vector1))   
            zzz_y = np.array([0])
            train_x = np.append(train_x,[zzz_x],axis=0)
            train_y = np.append(train_y,[zzz_y],axis=0)
    return train_x, train_y

In [286]:
num_elements_to_change = 30
size_ref_vector = 1000
label_1_matrix = np.random.randint(0,size_ref_vector-1, size=(40,num_elements_to_change) )
label_0_matrix = np.random.randint(0,size_ref_vector-1, size=(40,num_elements_to_change) )

label_1_sign = np.random.choice([-1,1], size=(40,num_elements_to_change) )
label_0_sign = np.random.choice([-1,1], size=(40,num_elements_to_change) )

print('label_1_matrix.shape: ' , label_1_matrix.shape)
print('label_0_matrix.shape: ' , label_0_matrix.shape)
print('label_1_sign.shape: ' , label_1_sign.shape)
print('label_0_sign.shape: ' , label_0_sign.shape)

Nset = 50
# Each set of dataset is Nset x 80
print('Generating the training dataset ...')
train_x, train_y=generate_dataset(Nset,size_ref_vector,label_1_matrix,label_0_matrix,label_1_sign,label_0_sign)
print(train_x.shape)
print(train_y.shape)

label_1_matrix.shape:  (40, 30)
label_0_matrix.shape:  (40, 30)
label_1_sign.shape:  (40, 30)
label_0_sign.shape:  (40, 30)
Generating the training dataset ...
(4000, 2000)
(4000, 1)


In [ ]:
# Each set of dataset is Nset x 80
print('Generating the testing dataset ...')
test_x, test_y=generate_dataset(25,size_ref_vector,label_1_matrix,label_0_matrix,label_1_sign,label_0_sign)
print(test_x.shape) # (2000, 2000)
print(test_y.shape) # (2000, 1)

Generating the testing dataset ...
(2000, 2000)
(2000, 1)


In [ ]:
# training

start_time = time.time()
history = ANN_model.fit(train_x, train_y, epochs=50, batch_size=64,validation_split=0.15,verbose = 2)
end_time = time.time()
exe_time = end_time - start_time
print("Execution time: ", exe_time)
scores = ANN_model.evaluate(test_x,test_y,verbose = 2)
print("Testing Accuracy = ", scores)

Epoch 1/50
54/54 - 1s - loss: 0.1016 - accuracy: 0.9679 - val_loss: 0.0166 - val_accuracy: 0.9917
Epoch 2/50
54/54 - 0s - loss: 0.0376 - accuracy: 0.9906 - val_loss: 0.0075 - val_accuracy: 1.0000
Epoch 3/50
54/54 - 0s - loss: 0.0232 - accuracy: 0.9950 - val_loss: 0.0087 - val_accuracy: 0.9950
Epoch 4/50
54/54 - 0s - loss: 0.0154 - accuracy: 0.9968 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 5/50
54/54 - 0s - loss: 0.0097 - accuracy: 0.9988 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 6/50
54/54 - 0s - loss: 0.0094 - accuracy: 0.9976 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 7/50
54/54 - 0s - loss: 0.0125 - accuracy: 0.9974 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 8/50
54/54 - 0s - loss: 0.0065 - accuracy: 0.9985 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 9/50
54/54 - 0s - loss: 0.0084 - accuracy: 0.9979 - val_loss: 5.6561e-04 - val_accuracy: 1.0000
Epoch 10/50
54/54 - 0s - loss: 0.0047 - accuracy: 0.9994 - val_loss: 5.0430e-04 - val_accuracy: 1.0000
Epoch 11/50

In [ ]:
# To get the results of the ANN using test dataset
y_pred = ANN_model.predict(test_x)

print('  max and min ')
print(np.max(y_pred))
print(np.min(y_pred))

epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

print(test_x.shape)
print(test_y.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(test_y, y_pred)
print(f"** Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
N=len(test_y)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(test_y,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
test_y_binary = test_y
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (test_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (test_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('Testing total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('2000 test cases: 1000 label 0; 1000 label 1')

  max and min 
0.9999943
2.025336e-05
(2000, 2000)
(2000, 1)
(2000, 1)
** CHECK Average BCE Loss for multiple samples: [0.0001]
Accuracy Score: 1.0
error0to1 =  0 ; label is 0
error1to0 =  0 ; label is 1
Testing total error =  0 percentError =  0.0
2000 test cases: 1000 label 0; 1000 label 1


In [290]:
print('test_x.shape ', test_x.shape)
print('test_y.shape ',test_y.shape)
print(y_pred_binary.shape)

print('Compare between test_y and y_pred_binary')

errorcase=[0]*80

for i in range(0,len(test_y)):
    index = (i % 80)
    if (test_y[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
print(errorcase)
print('Sum of errors = ' ,np.sum(errorcase))

test_x.shape  (2000, 2000)
test_y.shape  (2000, 1)
(2000, 1)
Compare between test_y and y_pred_binary
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sum of errors =  0


In [291]:
# To get the results of the ANN using train dataset
y_pred = ANN_model.predict(train_x)

print(train_x.shape)
print(train_y.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(train_y, y_pred)
print(f"** CHECK Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

N = len(train_y)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(train_y,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
train_y_binary = train_y
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (train_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (train_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('Training total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('train cases: ', len(train_y))

(4000, 2000)
(4000, 1)
(4000, 1)
** CHECK Average BCE Loss for multiple samples: [0.0005]
Accuracy Score: 0.99975
error0to1 =  1 ; label is 0
error1to0 =  0 ; label is 1
Training total error =  1 percentError =  0.025
train cases:  4000


In [269]:
print(label_1_matrix)

[[  1 323 177 ... 543 482 526]
 [702 197 859 ... 363 193 356]
 [313 253 407 ... 323 510 157]
 ...
 [737  69 738 ... 918  32 893]
 [647 697 271 ... 621   1  17]
 [853 299 323 ... 428 438 636]]


In [111]:
# DELETE below
# Part 2 of investigation
# 
#
DBset = np.empty((0,8))

for i in range(0,2000):
    #Ref vector
    aa=np.mean(train_x[i,0:2000])
    bb=np.mean(train_x[i,2000:4000])
    cc=np.mean(train_x[i,4000:6000])
    dd=np.mean(train_x[i,6000:8000])
    #Variant vector
    ee = np.mean(train_x[i,8192:10192])
    ff = np.mean(train_x[i,10192:12192])
    gg = np.mean(train_x[i,12192:14192])
    hh = np.mean(train_x[i,14192:16192])
    
    index = (i % 8)
    # index 0,1,2,3 are labe 1; index 4,5,6,7 are labe 0; 
    lowerR = 1.1
    upperR = 1.45
    
    if (index == 0): 
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh*kk])
    if (index == 1):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh])
    if (index == 2):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh*kk])
    if (index == 3):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff*kk, gg*kk, hh*kk])   
        
    if (index == 4):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg*kk, hh])
    if (index == 5):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg, hh])
    if (index == 6):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg*kk, hh])
    if (index == 7):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh])  
    
    DBset = np.append(DBset,[zzz],axis=0)
    # np.set_printoptions(precision=4)
    # print(zzz)
    

 


[[0.3039 0.3006 0.3049 ... 0.3006 0.3049 0.4289]
 [0.3045 0.3033 0.3086 ... 0.3599 0.3086 0.3018]
 [0.3049 0.3029 0.3019 ... 0.3978 0.3019 0.4091]
 ...
 [0.3108 0.3029 0.2993 ... 0.3029 0.2993 0.3026]
 [0.3005 0.3072 0.3018 ... 0.3072 0.4452 0.3074]
 [0.3052 0.3038 0.3063 ... 0.3038 0.3062 0.2989]]


In [128]:
# Create the test set of 2000

DBset_test = np.empty((0,8))

for i in range(0,2000):
    #Ref vector
    aa=np.mean(test_x[i,0:2000])
    bb=np.mean(test_x[i,2000:4000])
    cc=np.mean(test_x[i,4000:6000])
    dd=np.mean(test_x[i,6000:8000])
    #Variant vector
    ee = np.mean(test_x[i,8192:10192])
    ff = np.mean(test_x[i,10192:12192])
    gg = np.mean(test_x[i,12192:14192])
    hh = np.mean(test_x[i,14192:16192])
    
    index = (i % 8)
    # index 0,1,2,3 are labe 1; index 4,5,6,7 are labe 0; 
    lowerR = 1.1
    upperR = 1.45
    
    if (index == 0): 
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh*kk])
    if (index == 1):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh])
    if (index == 2):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh*kk])
    if (index == 3):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff*kk, gg*kk, hh*kk])   
        
    if (index == 4):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg*kk, hh])
    if (index == 5):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg, hh])
    if (index == 6):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg*kk, hh])
    if (index == 7):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh])  
    
    DBset_test = np.append(DBset_test,[zzz],axis=0)
    # np.set_printoptions(precision=4)
    # print(zzz)
    

 


In [ ]:
# Do not run
# debug
#
DBset_test = np.empty((0,8))

for i in range(0,2000):
    #Ref vector
    aa=np.mean(test_x[i,0:2000])
    bb=np.mean(test_x[i,2000:4000])
    cc=np.mean(test_x[i,4000:6000])
    dd=np.mean(test_x[i,6000:8000])
    #Variant vector
    ee = np.mean(test_x[i,8192:10192])
    ff = np.mean(test_x[i,10192:12192])
    gg = np.mean(test_x[i,12192:14192])
    hh = np.mean(test_x[i,14192:16192])
    
    index = (i % 8)
    # index 0,1,2,3 are labe 1; index 4,5,6,7 are labe 0; 
    lowerR = 1.1
    upperR = 1.45
    
    if (index == 0): 
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh*kk])
    if (index == 1):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh])
    if (index == 2):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff*kk, gg, hh*kk])
    if (index == 3):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff*kk, gg*kk, hh*kk])   
        
    if (index == 4):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg*kk, hh])
    if (index == 5):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg, hh])
    if (index == 6):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee*kk, ff, gg*kk, hh])
    if (index == 7):
        kk = random.uniform(lowerR,upperR)
        zzz = np.array([aa, bb, cc, dd, ee, ff, gg, hh])  
    
    DBset_test = np.append(DBset_test,[zzz],axis=0)
    # np.set_printoption

In [129]:
    
    print(DBset.shape)
    DBy = train_y[0:2000]
    print(DBy.shape)
    
    print(DBy[0:16])
    
    print(DBset_test.shape)
        
    
    

(2000, 8)
(2000,)
[1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
(2000, 8)


In [130]:
import os
DIR = "C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code"
out_path = os.path.join(DIR, "trainX_vectors3.csv")
print(out_path)
# DB_train.tofile(out_path,sep=',',fmt='%.5f')
np.savetxt(out_path,DBset,delimiter=',',fmt='%.5f')
print("Finished Export of Training cases ")

C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code\trainX_vectors3.csv
Finished Export of Training cases 


In [131]:
out_path = os.path.join(DIR, "trainY_vectors3.csv")
print(out_path)
# DB_test.tofile(out_path,sep=',',fmt='#.5f')
np.savetxt(out_path,DBy,delimiter=',',fmt='%.5f')
print("Finished Export of Testing cases ")

C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code\trainY_vectors3.csv
Finished Export of Testing cases 


In [132]:
out_path = os.path.join(DIR, "testX_vectors3.csv")
print(out_path)
# DB_train.tofile(out_path,sep=',',fmt='%.5f')
np.savetxt(out_path,DBset_test,delimiter=',',fmt='%.5f')
print("Finished Export of Training cases ")

C:/Users/gpang/iCloudDrive/Naja/20250628-Evo2/code\testX_vectors3.csv
Finished Export of Training cases 


In [133]:
def baseline_model3():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim = 8))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model


ANN_model = baseline_model3()

ANN_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 32)                288       
_________________________________________________________________
batch_normalization_22 (Batc (None, 32)                128       
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 8)                 264       
_________________________________________________________________
batch_normalization_23 (Batc (None, 8)                 32        
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 9         
Total params: 721
Trainable params: 641
Non-trainable params: 80
_______________________________________________________

In [150]:
start_time = time.time()

history = ANN_model.fit(DBset, DBy, epochs=40, batch_size=64,validation_split=0.2,verbose = 2)

end_time = time.time()
exe_time = end_time - start_time
print("Execution time: ", exe_time)


Epoch 1/40
25/25 - 1s - loss: 0.8202 - accuracy: 0.4313 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 2/40
25/25 - 0s - loss: 0.5825 - accuracy: 0.7013 - val_loss: 0.6846 - val_accuracy: 0.5000
Epoch 3/40
25/25 - 0s - loss: 0.4539 - accuracy: 0.8669 - val_loss: 0.6780 - val_accuracy: 0.5000
Epoch 4/40
25/25 - 0s - loss: 0.3591 - accuracy: 0.9306 - val_loss: 0.6705 - val_accuracy: 0.5000
Epoch 5/40
25/25 - 0s - loss: 0.2948 - accuracy: 0.9444 - val_loss: 0.6604 - val_accuracy: 0.5000
Epoch 6/40
25/25 - 0s - loss: 0.2442 - accuracy: 0.9494 - val_loss: 0.6488 - val_accuracy: 0.5000
Epoch 7/40
25/25 - 0s - loss: 0.1802 - accuracy: 0.9731 - val_loss: 0.6314 - val_accuracy: 0.5000
Epoch 8/40
25/25 - 0s - loss: 0.1544 - accuracy: 0.9825 - val_loss: 0.6261 - val_accuracy: 0.5000
Epoch 9/40
25/25 - 0s - loss: 0.1425 - accuracy: 0.9750 - val_loss: 0.6233 - val_accuracy: 0.5000
Epoch 10/40
25/25 - 0s - loss: 0.1214 - accuracy: 0.9825 - val_loss: 0.6233 - val_accuracy: 0.5000
Epoch 11/40
25/25 -

In [168]:
print('Ref vector  : ---   ---   ---   --- ')
print('Var index 0 : ---   ---   ---   -C- ', ' label 1')
print('Var index 1 : ---   -C-   ---   --- ', ' label 1')
print('Var index 2 : ---   -C-   ---   -C- ', ' label 1')
print('Var index 3 : -C-   -C-   -C-   -C- ', ' 70% label 1')

print('Var index 4 : ---   ---   -C-   --- ', ' label 0')
print('Var index 5 : -C-   ---   ---   --- ', ' label 0')
print('Var index 6 : -C-   ---   -C-   --- ', ' label 0')
print('Var index 7 : -C-   -C-   -C-   -C- ', ' 0.3% label 0')

print(DBset_test.shape)
print(DBy.shape)
print(DBy[0:16])
print(DBset_test[0:8,0:8])

Ref vector  : ---   ---   ---   --- 
Var index 0 : ---   ---   ---   -C-   label 1
Var index 1 : ---   -C-   ---   ---   label 1
Var index 2 : ---   -C-   ---   -C-   label 1
Var index 3 : -C-   -C-   -C-   -C-   70% label 1
Var index 4 : ---   ---   -C-   ---   label 0
Var index 5 : -C-   ---   ---   ---   label 0
Var index 6 : -C-   ---   -C-   ---   label 0
Var index 7 : -C-   -C-   -C-   -C-   0.3% label 0
(2000, 8)
(2000,)
[1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
[[0.2998 0.3119 0.3061 0.3032 0.2998 0.3119 0.3061 0.4311]
 [0.3094 0.3061 0.3012 0.3067 0.3094 0.374  0.3012 0.3067]
 [0.3065 0.3079 0.3024 0.3035 0.3065 0.4077 0.3024 0.4018]
 [0.3021 0.3061 0.3095 0.3106 0.3375 0.3403 0.3473 0.3523]
 [0.3054 0.3114 0.3052 0.2988 0.3054 0.3114 0.3381 0.2988]
 [0.3064 0.3077 0.3141 0.3042 0.3998 0.3077 0.3141 0.3042]
 [0.3043 0.304  0.3102 0.3022 0.4186 0.304  0.42   0.3022]
 [0.3065 0.3089 0.3051 0.3037 0.3065 0.3091 0.3052 0.3035]]


In [155]:
# To get the results of the ANN using test dataset
y_pred = ANN_model.predict(DBset_test)

print('  max and min ')
print(np.max(y_pred))
print(np.min(y_pred))

epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

print(DBset_test.shape)
print(DBy.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(DBy, y_pred)
print(f"** CHECK Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
N=len(DBy)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(DBy,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
test_y_binary = DBy
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (test_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (test_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('2000 test cases: 1000 label 0; 1000 label 1')

  max and min 
0.9997665
0.092390984
(2000, 8)
(2000,)
(2000, 1)
** CHECK Average BCE Loss for multiple samples: [0.2466]
Accuracy Score: 0.865
error0to1 =  270 ; label is 0
error1to0 =  0 ; label is 1
total error =  270 percentError =  13.5
2000 test cases: 1000 label 0; 1000 label 1


In [169]:
print(DBset_test.shape)
print(DBy.shape)
print(DBy[0:16])
#print(y_pred_binary.shape)
#print(y_pred_binary[0:16])

print('Compare between DBy and y_pred_binary')

errorcase=[0, 0, 0, 0, 0, 0, 0, 0]

for i in range(0,2000):
    index = (i % 8)
    if (index == 0 and DBy[i] != y_pred_binary[i] ): 
        errorcase[index] = errorcase[index]+1
    if (index == 1 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 2 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 3 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 4 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 5 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 6 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 7 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
print(errorcase)

(2000, 8)
(2000,)
[1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
Compare between DBy and y_pred_binary
[0, 0, 0, 0, 12, 11, 0, 247]


In [ ]:
# To check based on the training dataset

In [170]:
# To get the results of the ANN using train dataset
y_pred = ANN_model.predict(DBset)

print('  max and min ')
print(np.max(y_pred))
print(np.min(y_pred))

epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities

print(DBset.shape)
print(DBy.shape)
print(y_pred.shape)

total_loss = binary_cross_entropy_check(DBy, y_pred)
print(f"** CHECK Average BCE Loss for multiple samples: {total_loss}")

# To check on accuracy
# first, convert the elements in y_pred so that negative becones epsilon, largest is 1 - epsilon
epsilon = 1e-15  # Small value to prevent log(0)
y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
N=len(DBy)
threshold = 0.5
y_pred_binary = (y_pred >= threshold).astype(int)

accuracy = accuracy_score(DBy,y_pred_binary)
print("Accuracy Score:",accuracy)

#Count error cases:
#
test_y_binary = DBy
error0to1 = 0  # test_y is 0
error1to0 = 0  # test_y is 1
for i in range(N):
    if (test_y_binary[i] == 0 and y_pred_binary[i] == 1):
        error0to1 += 1
    if (test_y_binary[i] == 1 and y_pred_binary[i] == 0):
        error1to0 += 1
        
print('error0to1 = ',error0to1, '; label is 0')
print('error1to0 = ',error1to0, '; label is 1')
print('total error = ',error0to1+error1to0,'percentError = ', 100*(error0to1+error1to0)/N)
print('2000 training cases: 1000 label 0; 1000 label 1')

  max and min 
0.9997596
0.09228331
(2000, 8)
(2000,)
(2000, 1)
** CHECK Average BCE Loss for multiple samples: [0.248]
Accuracy Score: 0.863
error0to1 =  274 ; label is 0
error1to0 =  0 ; label is 1
total error =  274 percentError =  13.7
2000 training cases: 1000 label 0; 1000 label 1


In [172]:
print('Training dataset: Compare between DBy and y_pred_binary')

errorcase=[0, 0, 0, 0, 0, 0, 0, 0]

for i in range(0,2000):
    index = (i % 8)
    if (index == 0 and DBy[i] != y_pred_binary[i] ): 
        errorcase[index] = errorcase[index]+1
    if (index == 1 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 2 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 3 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 4 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 5 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 6 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
    if (index == 7 and DBy[i] != y_pred_binary[i]): 
        errorcase[index] = errorcase[index]+1
print(errorcase)
print(np.sum(errorcase))

Training dataset: Compare between DBy and y_pred_binary
[0, 0, 0, 0, 20, 8, 0, 246]
274


In [40]:
# NO USE BELOW

#Define ANOTHER ANN model
input_dim = 16384
ANNmodel2 = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(1500,input_dim)),
    layers.Dense(128, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Output layer for binary classification
])

# compile with cross entropy
ANNmodel2.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
# training

start_time = time.time()

history = ANNmodel2.fit(x_train, y_train, epochs=30, batch_size=32,verbose = 2)

end_time = time.time()
exe_time = end_time - start_time
print("Execution time: ", exe_time)

In [ ]:
scores = ANNmodel2.evaluate(test_x,test_y,verbose = 2)

In [55]:
# test code
import numpy as np
from keras.losses import binary_crossentropy
import keras.backend as K

# Example true labels and predicted probabilities
y_true = np.array([0, 1, 1, 0, 1])
y_pred = np.array([0.1, 0.9, 0.8, 0.2, 0.7])

# Compute Binary Cross-Entropy using NumPy
def binary_cross_entropy(y_true, y_pred):
    bce = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return bce

bce_loss = binary_cross_entropy(y_true, y_pred)
print(f"***Binary Cross-Entropy Loss (function): {bce_loss}")

#===========================================================================
def binary_cross_entropy_np(y_true, y_pred):
    """
    Calculates Binary Cross-Entropy loss for multiple samples using NumPy.
    y_true: NumPy array of actual labels (0s and 1s)
    y_pred: NumPy array of predicted probabilities (between 0 and 1)
    """
    epsilon = 1e-15  # Small value to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip probabilities
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

total_loss = binary_cross_entropy_np(y_true, y_pred)
print(f"--Average BCE Loss for multiple samples: {total_loss}")
#--------------------------------------------------------------------------

# Compute Binary Cross-Entropy using Keras
# DOES NOT WORK
# bce_loss_keras = binary_crossentropy(K.constant(y_true), K.constant(y_pred)).numpy()
# print(f"Binary Cross-Entropy Loss (Keras): {bce_loss_keras}")

***Binary Cross-Entropy Loss (function): 0.20273661557656092
--Average BCE Loss for multiple samples: 0.20273661557656092


In [84]:
aaa = np.array([0, 1, 1, -2.5, 1 , 3.4])
bbb = np.array([0.1, 0.9, 0.8, 0.2, 0.7])

print(max(aaa))
print(min(aaa))

3.4
-2.5


In [ ]:
    DBset = np.empty((0,4))
    DBset_tmp = np.array([1,2,3,4])
    DBset = np.append(DBset,[DBset_tmp],axis=0)
    print(DBset)
    
    DBset_tmp = np.array([5,6,7,8])
    DBset = np.append(DBset,[DBset_tmp],axis=0)

    print(DBset)
    print('========================')
    DBset_tmp = np.array([9,10,11,12])
    DBset = np.append(DBset,[DBset_tmp],axis=0)

    print(DBset)

In [ ]:
# NO NEED TO LOAD / READIN

# READ IN CSV (takes 30 seconds)
new_train_values = np.loadtxt("train_vectors2.csv",delimiter=",")
print(new_train_values.shape)
new_train = new_train_values.reshape((4000,16385))
print('new_train.shape = ',new_train.shape)


new_test_values = np.loadtxt("test_vectors2.csv",delimiter=",")
print(new_test_values.shape)
new_test = new_test_values.reshape((4000,16385))
print('new_test.shape = ',new_test.shape)

train_x = new_train[:,0:16384] 
train_y = new_train[:,16384]
print(train_x.shape)
print(train_y.shape)
#===================
test_x = new_test[:,0:16384] 
test_y = new_test[:,16384]
print(test_x.shape)
print(test_y.shape)